# 语料数据分析

In [1]:
import os
import json
import jieba
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
%matplotlib inline

In [3]:
BASE_DIR = os.path.dirname(os.path.dirname(os.path.abspath('__file__')))
NEWS_DATASET_PATH = os.path.join(BASE_DIR, 'text_summ/datasets/sqlResult_1558435.csv')

In [4]:
BASE_DIR

'/home/yy/Code/github/text-summarization'

## 新闻语料数据

In [5]:
news_dataset = pd.read_csv(NEWS_DATASET_PATH)

In [6]:
news_dataset.head(3)

,id,author,source,content,feature,title,url
0,89617,NaN,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""...",小米MIUI 9首批机型曝光：共计15款,http://www.cnbeta.com/articles/tech/623597.htm
1,89616,NaN,快科技@http://www.kkj.cn/,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""15""...",骁龙835在Windows 10上的性能表现有望改善,http://www.cnbeta.com/articles/tech/623599.htm
2,89615,NaN,快科技@http://www.kkj.cn/,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\n至于...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""18""...",一加手机5细节曝光：3300mAh、充半小时用1天,http://www.cnbeta.com/articles/tech/623601.htm


News datasets features

In [7]:
features = news_dataset['feature']

feature_json = []
for feature in features:
    feature_json.append(json.loads(feature))

news_features = pd.DataFrame(feature_json)

In [8]:
news_features.head(1)

,type,site,commentNum,joinNum,clickNum,shareNum,url,site_sign,label,summary,...,likeNum,keywords,rich_label,all_type,issuing_department,id,no,word_count,related_areas,reported_area
0,科技,cnbeta,37,20007,19920,0,http://www.cnbeta.com/articles/tech/623597.htm,cnbeta,,MIUI 9或许真的要来了！日前，微博数码博主@罗逸文V曝光了MIUI9首批支持机型，共有1...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
print(news_dataset['content'][1])

骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考虑性能而去屏蔽掉小核心。相反，他们正联手微软，找到一种适合桌面平台的、兼顾性能和功耗的完美方案。
报道称，微软已经拿到了一些新的源码，以便Windows 10更好地理解big.little架构。
资料显示，骁龙835作为一款集成了CPU、GPU、基带、蓝牙/Wi-Fi的SoC，比传统的Wintel方案可以节省至少30%的PCB空间。
按计划，今年Q4，华硕、惠普、联想将首发骁龙835 Win10电脑，预计均是二合一形态的产品。
当然，高通骁龙只是个开始，未来也许还能见到三星Exynos、联发科、华为麒麟、小米澎湃等进入Windows 10桌面平台。



In [13]:
content = news_dataset['content'].iloc[-1]
print('\n'.join(content.split('\\n')))

　　新华社德国杜塞尔多夫６月６日电题：乒乓女球迷　
　　新华社记者王子江、张寒
　　熊老师离开上海前，特意花一千多元买了一只张继科代言的球拍，准备在世界锦标赛期间他赢了比赛后，能在球拍上签个名。但没想到张继科在３２强赛中，输给了韩国的李尚洙。
　　“拍子算是白拿过来了。”熊老师摇头，“就算是信仰充值了。”
　　“信仰充值”是乒乓球粉丝圈非常流行的词汇，简单来说，就是为自己喜欢的明星或球队花钱。
　　“熊老师”是这位２２岁的成都姑娘的“艺名”，“艺名”这一称呼也是她们自己形容的。熊老师的真实身份是上海一所名牌大学会计专业的大四学生，其他同学正在抓紧毕业找工作，她一个人溜到德国观看世乒赛。尽管带着张继科的球拍，但熊老师并不是张的铁粉，她并不喜欢张继科输了比赛之后，一堆“科迷”在赛场外抱头痛哭的场景，而显得比较理智。但是因为“科迷”数量众多，让她这种散户，也有些说不出的羡慕。
　　“他们竟然连续两天买断了一面看台上所有的门票，为的是挂起带有张继科名字的旗子，两天就要花４、５万人民币。”熊老师直喊过分。
　　熊老师与其他１０名中国队女球迷挤在一套三居室里，厨房的柜子里塞了很多方便面。至少三个人挤在一个房间里，条件的确差了点，但住宿费平摊下来每天只有大约２００元人民币，这样让大家心里平衡了很多。
　　房间里的所有人都是在网上认识的，大家也并不全知道对方真实的姓名和身份。“我们都用艺名。”来自广州的茶茶说。茶茶已经工作了几年，这次也是利用假期出来的。
　　所有人都是提前大约半年从国际乒联的官网上买了比赛的通票，可以观看全程所有的比赛，票价大约两千块人民币。５月底６月初并不是旅游旺季，机票也不是太贵，这让她们都很庆幸。
　　里面最幸运的应该是井井，她这次来的身份是一家时尚媒体的记者，她申请到了记者证，每天背着一个大相机在场地里边拍照边看球。她提前大约一个星期就来到了德国，比赛开始前她去附近几个城市玩了玩，这一趟世乒赛之旅非常充实。
　　十几个人中最不像球迷的是一位来自香港的女大学生，看上去文静内向，其他人都叫她香港妹。香港妹与熊老师一样，都是学的会计专业。她告诉父母这次出来是到德国看朋友，住宿和球票的花费都是平时做家教挣的，父母支援了机票钱，从香港到杜塞尔多夫往返只有三千多元人民币，让其他内地球迷非常羡慕。
　　香港妹以前并不懂乒乓球，去年里约奥运会时，一家人边吃饭边看电视，正好

### 清洗数据

In [16]:
news_dataset.head(3)

,id,author,source,content,feature,title,url
0,89617,NaN,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""...",小米MIUI 9首批机型曝光：共计15款,http://www.cnbeta.com/articles/tech/623597.htm
1,89616,NaN,快科技@http://www.kkj.cn/,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""15""...",骁龙835在Windows 10上的性能表现有望改善,http://www.cnbeta.com/articles/tech/623599.htm
2,89615,NaN,快科技@http://www.kkj.cn/,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\n至于...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""18""...",一加手机5细节曝光：3300mAh、充半小时用1天,http://www.cnbeta.com/articles/tech/623601.htm


In [17]:
title_conent_df = news_dataset.loc[:, ['title', 'content']]

In [192]:
import re

def strip_str(string):
    strip_symbols = ['，', '。', '《', '》', ' ', '!', '！' '、', '.', '；', '~', 
                     '、', '”', '“', ';', '%', '\n', '\\n', '（', '）', '/', '——',
                    '：']
    for symbol in strip_symbols:
        string = string.replace(symbol, '')

    return string

In [195]:
s = news_dataset['content'].iloc[-1]
for l in re.split(r'\\n|\n', s):
    l = re.sub(re.compile(r'\\u\d{4}'), '', '%r'%l)[1:-1]
    print(l)


新华社德国杜塞尔多夫６月６日电题：乒乓女球迷
新华社记者王子江、张寒
熊老师离开上海前，特意花一千多元买了一只张继科代言的球拍，准备在世界锦标赛期间他赢了比赛后，能在球拍上签个名。但没想到张继科在３２强赛中，输给了韩国的李尚洙。
“拍子算是白拿过来了。”熊老师摇头，“就算是信仰充值了。”
“信仰充值”是乒乓球粉丝圈非常流行的词汇，简单来说，就是为自己喜欢的明星或球队花钱。
“熊老师”是这位２２岁的成都姑娘的“艺名”，“艺名”这一称呼也是她们自己形容的。熊老师的真实身份是上海一所名牌大学会计专业的大四学生，其他同学正在抓紧毕业找工作，她一个人溜到德国观看世乒赛。尽管带着张继科的球拍，但熊老师并不是张的铁粉，她并不喜欢张继科输了比赛之后，一堆“科迷”在赛场外抱头痛哭的场景，而显得比较理智。但是因为“科迷”数量众多，让她这种散户，也有些说不出的羡慕。
“他们竟然连续两天买断了一面看台上所有的门票，为的是挂起带有张继科名字的旗子，两天就要花４、５万人民币。”熊老师直喊过分。
熊老师与其他１０名中国队女球迷挤在一套三居室里，厨房的柜子里塞了很多方便面。至少三个人挤在一个房间里，条件的确差了点，但住宿费平摊下来每天只有大约２００元人民币，这样让大家心里平衡了很多。
房间里的所有人都是在网上认识的，大家也并不全知道对方真实的姓名和身份。“我们都用艺名。”来自广州的茶茶说。茶茶已经工作了几年，这次也是利用假期出来的。
所有人都是提前大约半年从国际乒联的官网上买了比赛的通票，可以观看全程所有的比赛，票价大约两千块人民币。５月底６月初并不是旅游旺季，机票也不是太贵，这让她们都很庆幸。
里面最幸运的应该是井井，她这次来的身份是一家时尚媒体的记者，她申请到了记者证，每天背着一个大相机在场地里边拍照边看球。她提前大约一个星期就来到了德国，比赛开始前她去附近几个城市玩了玩，这一趟世乒赛之旅非常充实。
十几个人中最不像球迷的是一位来自香港的女大学生，看上去文静内向，其他人都叫她香港妹。香港妹与熊老师一样，都是学的会计专业。她告诉父母这次出来是到德国看朋友，住宿和球票的花费都是平时做家教挣的，父母支援了机票钱，从香港到杜塞尔多夫往返只有三千多元人民币，让其他内地球迷非常羡慕。
香港妹以前并不懂乒乓球，去年里约奥运会时，一家人边吃饭边看电视，正好电视直播乒乓球比赛，从此她就成了中国队的球迷。
香港妹

处理清洗数据

In [115]:
def drop_na_rows(dataset, columns=('title', 'content')):
    """
    drop na rows.

    :param dataset, pandas DataFrame
    :return drop na dataset, pandas DataFrame
    """

    for col in columns:
        dataset = dataset[dataset[col].notnull()]

    return dataset

输出到文件

In [197]:
NEWS_CORPUS_OUTPUT_PATH = os.path.join(BASE_DIR, 'text_summ/datasets/news_corpus.csv')

In [290]:
title_content_df = drop_na_rows(news_dataset).loc[:, ['title', 'content']]

In [291]:
title_content_df.head(1)

,title,content
0,小米MIUI 9首批机型曝光：共计15款,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...


In [325]:
def cleanup_data(s):
    s = str(s)
    news_lines = []
    for l in re.split(r'\\n|\n', s):
        l = re.sub(re.compile(r'\\u\d{4}'), '', '%r'%l)[1:-1]
        if l:
            news_lines.append(l.strip())

    return '\n'.join(news_lines)

In [319]:
title_content_array = []
for title, content in title_conent_df.values:
    title = cleanup_data(title)
    content = cleanup_data(content)
    title_content_array.append([title, content])

cleanded_title_content_df = pd.DataFrame(title_content_array, columns=['title', 'content'])

In [422]:
news_dataset.loc[:, ['title', 'content']]

,title,content
0,小米MIUI 9首批机型曝光：共计15款,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...
1,骁龙835在Windows 10上的性能表现有望改善,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...
2,一加手机5细节曝光：3300mAh、充半小时用1天,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\n至于...
3,葡森林火灾造成至少62人死亡 政府宣布进入紧急状态（组图）,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\n
4,44岁女子约网友被拒暴雨中裸奔 交警为其披衣相随,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\n@深圳交警微博称：昨日清晨交...
...,...,...
89606,（体育）（2）冰球——国家女子冰球队海外选秀在多伦多展开,新华社照片，多伦多，2017年6月7日\n（体育）（2）冰球——国家女子冰球队海外选秀在多伦...
89607,（经济）甘肃集中开工35个重点交通建设项目,新华社兰州6月3日电（王衡、徐丹）记者从甘肃省交通运输厅获悉，甘肃近日集中开建高速公路、普通...
89608,法国议会选举 马克龙有望获“压倒性胜利”,\n\n2017年5月29日，在法国巴黎郊外的凡尔赛宫，法国总统马克龙出席新闻发布会。（新华...
89609,哈佛大学为何取消这些新生入选资格？,\n\n2017年5月25日，在美国马萨诸塞州剑桥市，哈佛大学毕业生在毕业典礼上欢呼。（新华...


In [425]:
def preprocess(df, columns=['title', 'content']):
    new_df = df.loc[:, columns]

    new_array = []
    for cols in new_df.values:
        new_cols = []
        for col in cols:
            new_cols.append(cleanup_data(col))

        new_array.append(new_cols)

    return pd.DataFrame(new_array, columns=columns)

In [418]:
# cleanded_title_content_df['content'].iloc[-1]

In [419]:
# title_content_df['content'].iloc[-1]

In [331]:
clenend_df = preprocess(news_dataset)

In [333]:
clenend_df.head()

,title,content
0,小米MIUI 9首批机型曝光：共计15款,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...
1,骁龙835在Windows 10上的性能表现有望改善,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...
2,一加手机5细节曝光：3300mAh、充半小时用1天,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\n至于...
3,葡森林火灾造成至少62人死亡 政府宣布进入紧急状态（组图）,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄
4,44岁女子约网友被拒暴雨中裸奔 交警为其披衣相随,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\n@深圳交警微博称：昨日清晨交...


In [341]:
df.to_csv?

In [342]:
df = preprocess(news_dataset)
df.to_csv(os.path.join(BASE_DIR, 'text_summ/datasets/news_corpus.csv'), index=False)

In [343]:
cleaned_dataset = pd.read_csv(os.path.join(BASE_DIR, 'text_summ/datasets/news_corpus.csv'))

In [344]:
cleaned_dataset.head()

,title,content
0,小米MIUI 9首批机型曝光：共计15款,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...
1,骁龙835在Windows 10上的性能表现有望改善,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...
2,一加手机5细节曝光：3300mAh、充半小时用1天,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\n至于...
3,葡森林火灾造成至少62人死亡 政府宣布进入紧急状态（组图）,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄
4,44岁女子约网友被拒暴雨中裸奔 交警为其披衣相随,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\n@深圳交警微博称：昨日清晨交...


分词

In [25]:
import jieba

In [26]:
def cut(s):
    return list(jieba.cut(s))

In [27]:
cut(strip_str(news_dataset['content'][0]))[:10]

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.742 seconds.
Prefix dict has been built succesfully.


['外自', '本周', '6', '月', '12', '日起', '除', '小米', '手机', '6']

In [36]:
news_dataset.head(2)

,id,author,source,content,feature,title,url
0,89617,NaN,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""...",小米MIUI 9首批机型曝光：共计15款,http://www.cnbeta.com/articles/tech/623597.htm
1,89616,NaN,快科技@http://www.kkj.cn/,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""15""...",骁龙835在Windows 10上的性能表现有望改善,http://www.cnbeta.com/articles/tech/623599.htm


In [378]:
news_corpus_dataset = pd.read_csv(os.path.join(BASE_DIR, 'text_summ/datasets/news_corpus.csv'))

In [379]:
news_corpus_dataset.head(3)

,title,content
0,小米MIUI 9首批机型曝光：共计15款,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...
1,骁龙835在Windows 10上的性能表现有望改善,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...
2,一加手机5细节曝光：3300mAh、充半小时用1天,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\n至于...


分词，解析成`gensim`的格式

In [431]:
STRIP_SYMBOLS = list(set(['，', '。', '《', '》', ' ', '!', '！' '、', '.', '；', '~', '、', 
                          '”', '“', ';', '%', '\n', '\\n', '（', '）', '/', '——', '：',
                          '…', '@', '！', '？', ' ,', ' :', '】', '’',')', '(', '／', '·',
                         '-', '?']))

In [367]:
strip_symbol_filter = lambda x: False if x in STRIP_SYMBOLS else True

In [374]:
list(filter(strip_symbol_filter, cut('小米MIUI 9首批机型曝光：共计15款 此外')))

['小米', 'MIUI', '9', '首批', '机型', '曝光', '共计', '15', '款', '此外']

In [404]:
strip_symbol_filter = lambda x: False if x in STRIP_SYMBOLS else True

for title, content in news_corpus_dataset.values:
    title_tokens = cut(title)
    content_tokens = cut(content)

    title_tokens = list(filter(strip_symbol_filter, title_tokens))
    content_tokens = list(filter(strip_symbol_filter, content_tokens))
    print(' '.join(title_tokens))
    break

小米 MIUI 9 首批 机型 曝光 共计 15 款


In [432]:
def corpus_to_file(df, filepath, debug=False):
    strip_symbol_filter = lambda x: False if x in STRIP_SYMBOLS else True

    counter = 0

    with open(filepath, 'w') as f:
        for title, content in df.values:
            title_tokens = cut(str(title))
            content_tokens = cut(str(content))

            title_tokens = list(filter(strip_symbol_filter, title_tokens))
            content_tokens = list(filter(strip_symbol_filter, content_tokens))

            f.write(' '.join(title_tokens) + '\n')
            f.write(' '.join(content_tokens) + '\n')

            counter += 1

            if debug:
                print('parse %s rows' % counter)

In [385]:
NEWS_CORPUS_DATASET_FILEPATH = os.path.join(BASE_DIR, 'text_summ/datasets/news_corpus.csv')
NEWS_CORPUS_FILEPATH = os.path.join(BASE_DIR, 'text_summ/datasets/news_corpus.txt')

In [405]:
news_corpus_dataset = pd.read_csv(NEWS_CORPUS_DATASET_FILEPATH)

In [427]:
# ' '.join(list(filter(strip_symbol_filter, cut(news_corpus_dataset['content'].iloc[4]))))

In [413]:
news_corpus_dataset.head()

,title,content
0,小米MIUI 9首批机型曝光：共计15款,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...
1,骁龙835在Windows 10上的性能表现有望改善,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...
2,一加手机5细节曝光：3300mAh、充半小时用1天,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\n至于...
3,葡森林火灾造成至少62人死亡 政府宣布进入紧急状态（组图）,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄
4,44岁女子约网友被拒暴雨中裸奔 交警为其披衣相随,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\n@深圳交警微博称：昨日清晨交...


In [433]:
corpus_to_file(news_corpus_dataset, NEWS_CORPUS_FILEPATH, debug=True)

parse 1 rows
parse 2 rows
parse 3 rows
parse 4 rows
parse 5 rows
parse 6 rows
parse 7 rows
parse 8 rows
parse 9 rows
parse 10 rows
parse 11 rows
parse 12 rows
parse 13 rows
parse 14 rows
parse 15 rows
parse 16 rows
parse 17 rows
parse 18 rows
parse 19 rows
parse 20 rows
parse 21 rows
parse 22 rows
parse 23 rows
parse 24 rows
parse 25 rows
parse 26 rows
parse 27 rows
parse 28 rows
parse 29 rows
parse 30 rows
parse 31 rows
parse 32 rows
parse 33 rows
parse 34 rows
parse 35 rows
parse 36 rows
parse 37 rows
parse 38 rows
parse 39 rows
parse 40 rows
parse 41 rows
parse 42 rows
parse 43 rows
parse 44 rows
parse 45 rows
parse 46 rows
parse 47 rows
parse 48 rows
parse 49 rows
parse 50 rows
parse 51 rows
parse 52 rows
parse 53 rows
parse 54 rows
parse 55 rows
parse 56 rows
parse 57 rows
parse 58 rows
parse 59 rows
parse 60 rows
parse 61 rows
parse 62 rows
parse 63 rows
parse 64 rows
parse 65 rows
parse 66 rows
parse 67 rows
parse 68 rows
parse 69 rows
parse 70 rows
parse 71 rows
parse 72 rows
p

parse 589 rows
parse 590 rows
parse 591 rows
parse 592 rows
parse 593 rows
parse 594 rows
parse 595 rows
parse 596 rows
parse 597 rows
parse 598 rows
parse 599 rows
parse 600 rows
parse 601 rows
parse 602 rows
parse 603 rows
parse 604 rows
parse 605 rows
parse 606 rows
parse 607 rows
parse 608 rows
parse 609 rows
parse 610 rows
parse 611 rows
parse 612 rows
parse 613 rows
parse 614 rows
parse 615 rows
parse 616 rows
parse 617 rows
parse 618 rows
parse 619 rows
parse 620 rows
parse 621 rows
parse 622 rows
parse 623 rows
parse 624 rows
parse 625 rows
parse 626 rows
parse 627 rows
parse 628 rows
parse 629 rows
parse 630 rows
parse 631 rows
parse 632 rows
parse 633 rows
parse 634 rows
parse 635 rows
parse 636 rows
parse 637 rows
parse 638 rows
parse 639 rows
parse 640 rows
parse 641 rows
parse 642 rows
parse 643 rows
parse 644 rows
parse 645 rows
parse 646 rows
parse 647 rows
parse 648 rows
parse 649 rows
parse 650 rows
parse 651 rows
parse 652 rows
parse 653 rows
parse 654 rows
parse 655 

parse 1166 rows
parse 1167 rows
parse 1168 rows
parse 1169 rows
parse 1170 rows
parse 1171 rows
parse 1172 rows
parse 1173 rows
parse 1174 rows
parse 1175 rows
parse 1176 rows
parse 1177 rows
parse 1178 rows
parse 1179 rows
parse 1180 rows
parse 1181 rows
parse 1182 rows
parse 1183 rows
parse 1184 rows
parse 1185 rows
parse 1186 rows
parse 1187 rows
parse 1188 rows
parse 1189 rows
parse 1190 rows
parse 1191 rows
parse 1192 rows
parse 1193 rows
parse 1194 rows
parse 1195 rows
parse 1196 rows
parse 1197 rows
parse 1198 rows
parse 1199 rows
parse 1200 rows
parse 1201 rows
parse 1202 rows
parse 1203 rows
parse 1204 rows
parse 1205 rows
parse 1206 rows
parse 1207 rows
parse 1208 rows
parse 1209 rows
parse 1210 rows
parse 1211 rows
parse 1212 rows
parse 1213 rows
parse 1214 rows
parse 1215 rows
parse 1216 rows
parse 1217 rows
parse 1218 rows
parse 1219 rows
parse 1220 rows
parse 1221 rows
parse 1222 rows
parse 1223 rows
parse 1224 rows
parse 1225 rows
parse 1226 rows
parse 1227 rows
parse 12

parse 1679 rows
parse 1680 rows
parse 1681 rows
parse 1682 rows
parse 1683 rows
parse 1684 rows
parse 1685 rows
parse 1686 rows
parse 1687 rows
parse 1688 rows
parse 1689 rows
parse 1690 rows
parse 1691 rows
parse 1692 rows
parse 1693 rows
parse 1694 rows
parse 1695 rows
parse 1696 rows
parse 1697 rows
parse 1698 rows
parse 1699 rows
parse 1700 rows
parse 1701 rows
parse 1702 rows
parse 1703 rows
parse 1704 rows
parse 1705 rows
parse 1706 rows
parse 1707 rows
parse 1708 rows
parse 1709 rows
parse 1710 rows
parse 1711 rows
parse 1712 rows
parse 1713 rows
parse 1714 rows
parse 1715 rows
parse 1716 rows
parse 1717 rows
parse 1718 rows
parse 1719 rows
parse 1720 rows
parse 1721 rows
parse 1722 rows
parse 1723 rows
parse 1724 rows
parse 1725 rows
parse 1726 rows
parse 1727 rows
parse 1728 rows
parse 1729 rows
parse 1730 rows
parse 1731 rows
parse 1732 rows
parse 1733 rows
parse 1734 rows
parse 1735 rows
parse 1736 rows
parse 1737 rows
parse 1738 rows
parse 1739 rows
parse 1740 rows
parse 17

parse 2197 rows
parse 2198 rows
parse 2199 rows
parse 2200 rows
parse 2201 rows
parse 2202 rows
parse 2203 rows
parse 2204 rows
parse 2205 rows
parse 2206 rows
parse 2207 rows
parse 2208 rows
parse 2209 rows
parse 2210 rows
parse 2211 rows
parse 2212 rows
parse 2213 rows
parse 2214 rows
parse 2215 rows
parse 2216 rows
parse 2217 rows
parse 2218 rows
parse 2219 rows
parse 2220 rows
parse 2221 rows
parse 2222 rows
parse 2223 rows
parse 2224 rows
parse 2225 rows
parse 2226 rows
parse 2227 rows
parse 2228 rows
parse 2229 rows
parse 2230 rows
parse 2231 rows
parse 2232 rows
parse 2233 rows
parse 2234 rows
parse 2235 rows
parse 2236 rows
parse 2237 rows
parse 2238 rows
parse 2239 rows
parse 2240 rows
parse 2241 rows
parse 2242 rows
parse 2243 rows
parse 2244 rows
parse 2245 rows
parse 2246 rows
parse 2247 rows
parse 2248 rows
parse 2249 rows
parse 2250 rows
parse 2251 rows
parse 2252 rows
parse 2253 rows
parse 2254 rows
parse 2255 rows
parse 2256 rows
parse 2257 rows
parse 2258 rows
parse 22

...

parse 84110 rows
parse 84111 rows
parse 84112 rows
parse 84113 rows
parse 84114 rows
parse 84115 rows
parse 84116 rows
parse 84117 rows
parse 84118 rows
parse 84119 rows
parse 84120 rows
parse 84121 rows
parse 84122 rows
parse 84123 rows
parse 84124 rows
parse 84125 rows
parse 84126 rows
parse 84127 rows
parse 84128 rows
parse 84129 rows
parse 84130 rows
parse 84131 rows
parse 84132 rows
parse 84133 rows
parse 84134 rows
parse 84135 rows
parse 84136 rows
parse 84137 rows
parse 84138 rows
parse 84139 rows
parse 84140 rows
parse 84141 rows
parse 84142 rows
parse 84143 rows
parse 84144 rows
parse 84145 rows
parse 84146 rows
parse 84147 rows
parse 84148 rows
parse 84149 rows
parse 84150 rows
parse 84151 rows
parse 84152 rows
parse 84153 rows
parse 84154 rows
parse 84155 rows
parse 84156 rows
parse 84157 rows
parse 84158 rows
parse 84159 rows
parse 84160 rows
parse 84161 rows
parse 84162 rows
parse 84163 rows
parse 84164 rows
parse 84165 rows
parse 84166 rows
parse 84167 rows
parse 84168 ro

parse 84791 rows
parse 84792 rows
parse 84793 rows
parse 84794 rows
parse 84795 rows
parse 84796 rows
parse 84797 rows
parse 84798 rows
parse 84799 rows
parse 84800 rows
parse 84801 rows
parse 84802 rows
parse 84803 rows
parse 84804 rows
parse 84805 rows
parse 84806 rows
parse 84807 rows
parse 84808 rows
parse 84809 rows
parse 84810 rows
parse 84811 rows
parse 84812 rows
parse 84813 rows
parse 84814 rows
parse 84815 rows
parse 84816 rows
parse 84817 rows
parse 84818 rows
parse 84819 rows
parse 84820 rows
parse 84821 rows
parse 84822 rows
parse 84823 rows
parse 84824 rows
parse 84825 rows
parse 84826 rows
parse 84827 rows
parse 84828 rows
parse 84829 rows
parse 84830 rows
parse 84831 rows
parse 84832 rows
parse 84833 rows
parse 84834 rows
parse 84835 rows
parse 84836 rows
parse 84837 rows
parse 84838 rows
parse 84839 rows
parse 84840 rows
parse 84841 rows
parse 84842 rows
parse 84843 rows
parse 84844 rows
parse 84845 rows
parse 84846 rows
parse 84847 rows
parse 84848 rows
parse 84849 ro

parse 85371 rows
parse 85372 rows
parse 85373 rows
parse 85374 rows
parse 85375 rows
parse 85376 rows
parse 85377 rows
parse 85378 rows
parse 85379 rows
parse 85380 rows
parse 85381 rows
parse 85382 rows
parse 85383 rows
parse 85384 rows
parse 85385 rows
parse 85386 rows
parse 85387 rows
parse 85388 rows
parse 85389 rows
parse 85390 rows
parse 85391 rows
parse 85392 rows
parse 85393 rows
parse 85394 rows
parse 85395 rows
parse 85396 rows
parse 85397 rows
parse 85398 rows
parse 85399 rows
parse 85400 rows
parse 85401 rows
parse 85402 rows
parse 85403 rows
parse 85404 rows
parse 85405 rows
parse 85406 rows
parse 85407 rows
parse 85408 rows
parse 85409 rows
parse 85410 rows
parse 85411 rows
parse 85412 rows
parse 85413 rows
parse 85414 rows
parse 85415 rows
parse 85416 rows
parse 85417 rows
parse 85418 rows
parse 85419 rows
parse 85420 rows
parse 85421 rows
parse 85422 rows
parse 85423 rows
parse 85424 rows
parse 85425 rows
parse 85426 rows
parse 85427 rows
parse 85428 rows
parse 85429 ro

parse 85963 rows
parse 85964 rows
parse 85965 rows
parse 85966 rows
parse 85967 rows
parse 85968 rows
parse 85969 rows
parse 85970 rows
parse 85971 rows
parse 85972 rows
parse 85973 rows
parse 85974 rows
parse 85975 rows
parse 85976 rows
parse 85977 rows
parse 85978 rows
parse 85979 rows
parse 85980 rows
parse 85981 rows
parse 85982 rows
parse 85983 rows
parse 85984 rows
parse 85985 rows
parse 85986 rows
parse 85987 rows
parse 85988 rows
parse 85989 rows
parse 85990 rows
parse 85991 rows
parse 85992 rows
parse 85993 rows
parse 85994 rows
parse 85995 rows
parse 85996 rows
parse 85997 rows
parse 85998 rows
parse 85999 rows
parse 86000 rows
parse 86001 rows
parse 86002 rows
parse 86003 rows
parse 86004 rows
parse 86005 rows
parse 86006 rows
parse 86007 rows
parse 86008 rows
parse 86009 rows
parse 86010 rows
parse 86011 rows
parse 86012 rows
parse 86013 rows
parse 86014 rows
parse 86015 rows
parse 86016 rows
parse 86017 rows
parse 86018 rows
parse 86019 rows
parse 86020 rows
parse 86021 ro

parse 86577 rows
parse 86578 rows
parse 86579 rows
parse 86580 rows
parse 86581 rows
parse 86582 rows
parse 86583 rows
parse 86584 rows
parse 86585 rows
parse 86586 rows
parse 86587 rows
parse 86588 rows
parse 86589 rows
parse 86590 rows
parse 86591 rows
parse 86592 rows
parse 86593 rows
parse 86594 rows
parse 86595 rows
parse 86596 rows
parse 86597 rows
parse 86598 rows
parse 86599 rows
parse 86600 rows
parse 86601 rows
parse 86602 rows
parse 86603 rows
parse 86604 rows
parse 86605 rows
parse 86606 rows
parse 86607 rows
parse 86608 rows
parse 86609 rows
parse 86610 rows
parse 86611 rows
parse 86612 rows
parse 86613 rows
parse 86614 rows
parse 86615 rows
parse 86616 rows
parse 86617 rows
parse 86618 rows
parse 86619 rows
parse 86620 rows
parse 86621 rows
parse 86622 rows
parse 86623 rows
parse 86624 rows
parse 86625 rows
parse 86626 rows
parse 86627 rows
parse 86628 rows
parse 86629 rows
parse 86630 rows
parse 86631 rows
parse 86632 rows
parse 86633 rows
parse 86634 rows
parse 86635 ro